In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle
import time
import os
import json
import seaborn as sns
import tensorflow as tf
import random
import cv2
import copy
import colorsys

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms

from tensorflow.keras.applications import VGG16,InceptionV3
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential

from keras.applications import VGG16
from keras.applications.resnet import ResNet152
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from datetime import datetime

from pandas import json_normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score,classification_report
from sklearn.preprocessing import normalize,StandardScaler,RobustScaler,MinMaxScaler
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.compose import ColumnTransformer



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.python.platform import gfile


from tensorflow import keras
from keras.utils import to_categorical
from keras import regularizers
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D
from keras.losses import binary_crossentropy

from scipy.stats import skew
from scipy.optimize import linear_sum_assignment

from tqdm import tqdm
from PIL import Image
from sklearn import metrics

from scipy.optimize import linear_sum_assignment as linear_assignment
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort import nn_matching
from deep_sort.tools import generate_detections as gdet

In [2]:
tf.config.run_functions_eagerly(True)

In [3]:
print(tf.executing_eagerly())

True


In [4]:
print(tf.__version__)

2.9.0


In [5]:
print(np.__version__)

1.24.3


In [6]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

# tf.compat.v1.disable_eager_execution()
# sess = tf.compat.v1.Session()

### Loading Trained InceptionV3 Model

In [7]:
model = tf.saved_model.load('models/savedModels/')

Metal device set to: Apple M1 Max


2023-05-29 08:29:25.658170: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-29 08:29:25.658590: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
graph_filename = "models/graphs/InceptionCustom/output_graph.pb"

In [9]:
model_path = graph_filename
graph = tf.Graph()
with graph.as_default():
    graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(model_path, 'rb') as f:
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

# Start a TensorFlow session
with tf.compat.v1.Session(graph=graph) as sess:
    # Get the output node names
    output_node_names = [node.name for node in graph_def.node if node.op == 'Placeholder']

2023-05-29 08:29:29.798216: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-29 08:29:29.798236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
loaded_model = model

with tf.summary.create_file_writer('logs/').as_default():
    # Enable tracing of TensorFlow operations
    tf.summary.trace_on(graph=True)

    # Execute a forward pass on the model to capture the graph
    inputs = tf.ones([1, 224, 224, 3])  # Provide a sample input
    _ = loaded_model(inputs)  # Forward pass

    # Export the captured graph to the log directory
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir='logs/')

# Launch TensorBoard with the specified log directory
%load_ext tensorboard
%tensorboard --logdir logs/


2023-05-29 08:29:29.972875: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-29 08:29:29.980748: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [11]:
def print_io(saved_model_dir):

    # Load the model
    loaded_model = tf.saved_model.load(saved_model_dir)

    # Get the input signature of the model
    input_signature = loaded_model.signatures['serving_default'].inputs
    print("\nInput Signatures: ")
    for input in input_signature:
        print(input.name)

    # Get the output signature of the model
    output_signature = loaded_model.signatures['serving_default'].outputs
    print("\nOutput Signatures: ")
    for output in output_signature:
        print(output.name)
                
print_io("models/savedModels/")


Input Signatures: 
inception_v3_input:0
unknown:0
unknown_0:0
unknown_1:0
unknown_2:0
unknown_3:0
unknown_4:0
unknown_5:0
unknown_6:0
unknown_7:0
unknown_8:0
unknown_9:0
unknown_10:0
unknown_11:0
unknown_12:0
unknown_13:0
unknown_14:0
unknown_15:0
unknown_16:0
unknown_17:0
unknown_18:0
unknown_19:0
unknown_20:0
unknown_21:0
unknown_22:0
unknown_23:0
unknown_24:0
unknown_25:0
unknown_26:0
unknown_27:0
unknown_28:0
unknown_29:0
unknown_30:0
unknown_31:0
unknown_32:0
unknown_33:0
unknown_34:0
unknown_35:0
unknown_36:0
unknown_37:0
unknown_38:0
unknown_39:0
unknown_40:0
unknown_41:0
unknown_42:0
unknown_43:0
unknown_44:0
unknown_45:0
unknown_46:0
unknown_47:0
unknown_48:0
unknown_49:0
unknown_50:0
unknown_51:0
unknown_52:0
unknown_53:0
unknown_54:0
unknown_55:0
unknown_56:0
unknown_57:0
unknown_58:0
unknown_59:0
unknown_60:0
unknown_61:0
unknown_62:0
unknown_63:0
unknown_64:0
unknown_65:0
unknown_66:0
unknown_67:0
unknown_68:0
unknown_69:0
unknown_70:0
unknown_71:0
unknown_72:0
unknown_73

In [12]:
print(model.signatures.keys())

KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, inception_v3_input) at 0x2DC93E8B0>}))


In [13]:
signature = model.signatures['serving_default']
print(signature.structured_outputs.keys())

dict_keys(['dense'])


### Loading Yolo

In [14]:
object_detection_model = cv2.dnn.readNet("models/yolov4-tiny.weights", "models/yolov4-tiny.cfg")
object_detection_model.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
object_detection_model.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)

### Reading in Video

In [15]:
video_path = 'Datasets/Video/QueenAnnRoy/moving/test.mov'
output_dir = 'Datasets/Video/QueenAnnRoy/stills_test/'
os.makedirs(output_dir, exist_ok=True)
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [16]:
classes = ['Subaru', 'car']
# with open("coco.names", "r") as f:
#     classes = [line.strip() for line in f.readlines()]

In [17]:
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
input_size = (244, 244)
input_scale = 1/255.0
confidence_thresh = .1
overlap = .1

In [18]:
classes

['Subaru', 'car']

In [19]:
cap

<VideoCapture 0x2bbba0b70>

### Defining Region of Interest (ROI)

In [20]:
roi_coordinates = [(352, 820), (855, 820), (1282, 270),(1082, 270)]

In [21]:
# Create a binary mask image with the same size as the original image
image = cv2.imread('Datasets/Video/QueenAnnRoy/stills/frame_000000.jpg')  # Load your image
mask = np.zeros_like(image)

# Convert the coords to numpy
roi_pts = np.array(roi_coordinates, dtype=np.int32)
cv2.fillPoly(mask, [roi_pts], (255, 255, 255))  # Set ROI pixels to white (255)

roi = cv2.bitwise_and(image, mask)

# cv2.imshow('ROI', roi)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

### Initializing DeepSort

##### Helper Functions

In [22]:
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, input_size)
    image = image / 255.0
    return image

In [23]:
def postprocess_boxes(pred_bbox, original_image, input_size, score_threshold, CLASSES):
    height, width = original_image.shape[:2]
    filtered_boxes = []
    for pred in pred_bbox:
        if np.any(np.greater(pred[4], score_threshold)):
            xmin = int(pred[0][0] * width)
            ymin = int(pred[0][1] * height)
            xmax = int(pred[0][2] * width)
            ymax = int(pred[0][3] * height)
            class_index = int(pred[0][5])
            class_name = CLASSES[class_index]
            score = float(pred[0][4])
            filtered_boxes.append([xmin, ymin, xmax, ymax, class_name, score])
    return filtered_boxes

In [24]:
def nms(boxes, overlap, method='nms'):
    if len(boxes) == 0:
        return []
    
    # Extract coordinates and scores
    x1 = boxes[:, 0].astype(float)
    y1 = boxes[:, 1].astype(float)
    x2 = boxes[:, 2].astype(float)
    y2 = boxes[:, 3].astype(float)
    scores = boxes[:, 5]

    # Calculate areas of bounding boxes
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Sort boxes by their scores in descending order
    order = scores.argsort()[::-1]

    # Initialize list for selected bounding boxes
    selected_boxes = []

    while order.size > 0:
        # Select box with the highest score
        max_idx = order[0]
        selected_boxes.append(boxes[max_idx])

        # Calculate intersection over union (IoU) with other boxes
        xx1 = np.maximum(x1[max_idx], x1[order[1:]])
        yy1 = np.maximum(y1[max_idx], y1[order[1:]])
        xx2 = np.minimum(x2[max_idx], x2[order[1:]])
        yy2 = np.minimum(y2[max_idx], y2[order[1:]])

        intersection = np.maximum(0.0, xx2 - xx1 + 1) * np.maximum(0.0, yy2 - yy1 + 1)
        iou = intersection / (areas[max_idx] + areas[order[1:]] - intersection)

        # Remove boxes with IoU higher than the threshold
        if method == 'nms':
            selected_indices = np.where(iou <= overlap)[0]
        elif method == 'soft-nms':
            weights = np.exp(-(iou * iou) / overlap)
            scores[order[1:]] *= weights
            selected_indices = np.where(scores[order[1:]] > confidence_thresh)[0]
        else:
            raise ValueError("Invalid method. Supported methods are 'nms' and 'soft-nms'.")

        order = order[selected_indices + 1]

    return selected_boxes

In [25]:
def draw_bbox(image, bboxes, CLASSES=classes, show_label=True, show_confidence = True, Text_colors=(255,255,0), rectangle_colors='', tracking=False):   
    NUM_CLASS = len(CLASSES)
    image_h, image_w, _ = image.shape
    hsv_tuples = [(1.0 * x / NUM_CLASS, 1., 1.) for x in range(NUM_CLASS)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)

    for i, bbox in enumerate(bboxes):
        coor = np.array(bbox[:4], dtype=np.int32)
        score = float(bbox[4])
        class_name = bbox[5]
        bbox_color = rectangle_colors if rectangle_colors != '' else colors[CLASSES.index(class_name)]
        bbox_thick = int(0.6 * (image_h + image_w) / 1000)
        if bbox_thick < 1: bbox_thick = 1
        fontScale = 0.75 * bbox_thick
        (x1, y1), (x2, y2) = (coor[0], coor[1]), (coor[2], coor[3])

        # put object rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), bbox_color, bbox_thick*2)

        if show_label:
            # get text label
            score_str = " {:.2f}".format(score) if show_confidence else ""

            if tracking: score_str = " "+str(score)

            label = "{}".format(class_name) + score_str

            # get text size
            (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                                                  fontScale, thickness=bbox_thick)
            # put filled text rectangle
            cv2.rectangle(image, (x1, y1), (x1 + text_width, y1 - text_height - baseline), bbox_color, thickness=cv2.FILLED)

            # put text above rectangle
            cv2.putText(image, label, (x1, y1-4), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale, Text_colors, bbox_thick, lineType=cv2.LINE_AA)

    return image

In [26]:
loaded_model = model
print(loaded_model.signatures)

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, inception_v3_input) at 0x2DC93E8B0>})


##### Object Tracking

In [27]:
def Object_tracking(object_detection_model, video_path, output_path, input_size=(416, 416), show=False, CLASSES=classes, score_threshold=0.3, iou_threshold=0.45, rectangle_colors='', Track_only=['cars', 'trucks']):
    # Definition of the parameters
    max_cosine_distance = 0.5
    nn_budget = None
    
    # Initializing tracker with Subaru CNN
    model_filename = "models/graphs/InceptionCustom/output_graph.pb"
    encoder = gdet.create_box_encoder(model_filename, output_name='Identity:0', batch_size=1)
    print(type(encoder))  # Should print "<class 'function'>"
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    times = []

    vid = cv2.VideoCapture(video_path)  # Read video frames

    # By default, VideoCapture returns float instead of int
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    codec = cv2.VideoWriter_fourcc(*'XVID')

    output_filename = "output.mov"
    output_path = os.path.join(output_path, output_filename)
    out = cv2.VideoWriter(output_path, codec, fps, (width, height))

    key_list = classes
    val_list = classes
    
    while True:
        ret, frame = vid.read()  # Read the next frame from the video
        print(frame.shape)
        if not ret:
            break  # End of video

        try:
            original_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
            original_image = original_image.astype(np.uint8)
 
        except:
            break

        image_data = preprocess_image(original_image)
        image_data = np.expand_dims(image_data, 0)
        
        image_data = np.transpose(image_data, (0, 3, 1, 2))
        image_data = image_data.astype(np.float32)
        
        t1 = time.time()
        
        object_detection_model.setInput(image_data)
        layer_outputs = object_detection_model.forward(object_detection_model.getUnconnectedOutLayersNames())

        # Process the output predictions
        pred_bbox = postprocess_boxes(layer_outputs, original_image, input_size, score_threshold, CLASSES)
        pred_bbox = np.array(pred_bbox)
        pred_bbox = nms(pred_bbox, iou_threshold, method='nms')

#         # Extract bboxes to boxes (x, y, width, height), scores and names
#         boxes, scores, names = [], [], []
#         for bbox in pred_bbox:
#             print("bbox print:",bbox)
#             class_index = int(float(bbox[5]))  # Convert the class index to an integer
#             if len(Track_only) != 0 and class_index < len(CLASSES) and CLASSES[class_index] in Track_only or len(Track_only) == 0:
#                 boxes.append([bbox[0], bbox[1], bbox[2], bbox[3]])
#                 scores.append(bbox[4])

        boxes, scores, names = [], [], []
        for bbox in pred_bbox:
            class_name = bbox[4]
            print("bbox print:",bbox)
            class_index = CLASSES.index(class_name)

            print(f"len(Track_only): {len(Track_only)}")
            print(f"class_index: {class_index}")
            print(f"len(CLASSES): {len(CLASSES)}")

            if class_index < len(CLASSES):
                print(f"CLASSES[class_index]: {CLASSES[class_index]}")
                print(f"CLASSES[class_index] in Track_only: {CLASSES[class_index] in Track_only}")

            if len(Track_only) != 0 and class_index < len(CLASSES) and CLASSES[class_index] in Track_only or len(Track_only) == 0:
                boxes.append([float(bbox[0]), float(bbox[1]), float(bbox[2]), float(bbox[3])])
                scores.append(bbox[5])
                names.append(bbox[4])

        # Obtain all the detections for the given frame
        boxes = np.array(boxes)
        print("boxes print",boxes)
        print("boxes shape",boxes.shape)
        print("boxes type",boxes.dtype)

        names = np.array(names)
        scores = np.array(scores)
        features = np.array(encoder(original_image, boxes))
        detections = [Detection(bbox, score, feature, class_name) for bbox, score, feature, class_name in
                  zip(boxes, scores, features, names)]

        # Pass detections to the deepsort object and obtain the track information
        tracker.predict()
        tracker.update(detections)

        # Obtain info from the tracks
        tracked_bboxes = []
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()  # Get the corrected/predicted bounding box
            class_name = track.get_class()  # Get the class name of the particular object
            tracking_id = track.track_id  # Get the ID for the particular track
            index = key_list[val_list.index(class_name)]  # Get the predicted object index by object name
            tracked_bboxes.append(bbox.tolist() + [tracking_id, index])  # Structure data that we could use with our draw_bbox function

        if len(times) > 0:
            ms = sum(times) / len(times) * 1000
            fps = 1000 / ms
        else:
            ms = 0
            fps = 0

        # Draw detection on frame
        image = draw_bbox(original_image, tracked_bboxes, CLASSES=CLASSES, tracking=True)
        image = cv2.putText(image, "Time: {:.1f} FPS".format(fps), (0, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,
                            (0, 0, 255), 2)

        if output_path != '':
            out.write(image)
        if show:
            cv2.imshow('output', image)
            if cv2.waitKey(25) & 0xFF == ord("q"):
                cv2.destroyAllWindows()
                break

    vid.release()
    cv2.destroyAllWindows()

In [ ]:
Object_tracking(object_detection_model,
                video_path,
                output_path=output_dir,
                input_size=(244, 244),
                show=True,
                CLASSES=classes,
                score_threshold=confidence_thresh,
                iou_threshold=overlap,
                rectangle_colors='',
                Track_only = ['Subaru', 'car']
               )

### Object Detection & Model Application

In [ ]:
with tqdm(total=frame_count, unit='frame') as pbar:
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Apply the ROI mask to the frame
        roi_frame = cv2.bitwise_and(frame, mask)

        # Perform object detection within the ROI
        blob = cv2.dnn.blobFromImage(roi_frame, input_scale, input_size, swapRB=True, crop=False)
        object_detection_model.setInput(blob)
        layer_outputs = object_detection_model.forward(object_detection_model.getUnconnectedOutLayersNames())

        # Process the bounding boxes
        boxes = []
        confidences = []
        class_ids = []
        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                class_name = classes[class_id]
                if class_name == 'car' and confidence > confidence_thresh:
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    width = int(detection[2] * frame.shape[1])
                    height = int(detection[3] * frame.shape[0])
                    x = int(center_x - width / 2)
                    y = int(center_y - height / 2)
                    boxes.append([x, y, width, height])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Apply non-maximum suppression
        indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_thresh, overlap)

        # Extract the detected object features for tracking
        features = np.array(boxes)  # Use bounding boxes as features (replace this with your feature extraction logic)

        # Create Detection instances
        detections = []
        for bbox, confidence, class_id in zip(boxes, confidences, class_ids):
            detection = Detection(bbox, confidence, class_id)
            detections.append(detection)

        # Perform object tracking
        outputs = tracker.update(features)

        for output, bbox in zip(outputs, boxes):
            track_id = output[4]

            # Draw bounding box and track ID on the frame
            x, y, w, h = bbox
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, str(track_id), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the processed frame
        cv2.imshow('Video', frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        pbar.update(1)

cap.release()
cv2.destroyAllWindows()